In [1]:
#Importing packages.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#message_df = pd.read_csv('https://raw.githubusercontent.com/erood/interviewqs.com_code_snippets/master/Datasets/sample_message_dataset.csv')

In [4]:
#message_df.to_csv('data/message.csv', index=False)

In [74]:
df = pd.read_csv('data/message.csv')

In [39]:
message_df = df.copy()
print(message_df.shape)
message_df.head(5)


(86, 4)


,date,timestamp,sender_id,receiver_id
0,2018-03-01,1519923378,1,5
1,2018-03-01,1519942810,1,4
2,2018-03-01,1519918950,1,5
3,2018-03-01,1519930114,1,2
4,2018-03-01,1519920410,1,2


In [40]:
#use sender_id and receiver_id to create primary key
message_df['key'] = np.where(message_df['sender_id'] >= message_df['receiver_id'],\
                            message_df['receiver_id'].astype(str)+message_df['sender_id'].astype(str),
                            message_df['sender_id'].astype(str)+message_df['receiver_id'].astype(str))

In [41]:
message_df.head(5)

,date,timestamp,sender_id,receiver_id,key
0,2018-03-01,1519923378,1,5,15
1,2018-03-01,1519942810,1,4,14
2,2018-03-01,1519918950,1,5,15
3,2018-03-01,1519930114,1,2,12
4,2018-03-01,1519920410,1,2,12


In [42]:
#sorting the dataframe base on primary key and then timestamp
message_df = message_df.sort_values(by=['key', 'timestamp'])

In [43]:
message_df.head(8)

,date,timestamp,sender_id,receiver_id,key
4,2018-03-01,1519920410,1,2,12
47,2018-03-01,1519920555,2,1,12
3,2018-03-01,1519930114,1,2,12
46,2018-03-01,1519930348,2,1,12
1,2018-03-01,1519942810,1,4,14
44,2018-03-01,1519943010,4,1,14
2,2018-03-01,1519918950,1,5,15
45,2018-03-01,1519920307,5,1,15


In [44]:
#lagging the key to use for which one should lag timestamp
message_df['key_lag'] = message_df['key'].shift(1)
#lagging the timstamp within the same key
message_df['tt_lag'] = np.where(message_df['key_lag']==message_df['key'],
                                message_df['timestamp'].shift(1),
                                0)
message_df['tt_lag'] = message_df['tt_lag'].astype(int)

In [45]:
#sender lag to compare if it is a respone
message_df['sender_lag'] = message_df['sender_id'].shift(1)
#label which one we receier id and sender lag
message_df['is_respone'] = np.where((message_df['receiver_id']==message_df['sender_lag']), 'yes', 'no')

In [46]:
#filter for messages sent and received by same id
message_df['same_sender_receiver'] = np.where(message_df['sender_id']==message_df['receiver_id'], 'yes', 'no')

#filtering out the same sender and receiver id, and senders have respone
message_df_filtered = message_df[(message_df.same_sender_receiver=='no') & (message_df.is_respone == 'yes')]

In [47]:
message_df_filtered.head(5)

,date,timestamp,sender_id,receiver_id,key,key_lag,tt_lag,sender_lag,is_respone,same_sender_receiver
47,2018-03-01,1519920555,2,1,12,12,1519920410,1.0,yes,no
3,2018-03-01,1519930114,1,2,12,12,1519920555,2.0,yes,no
46,2018-03-01,1519930348,2,1,12,12,1519930114,1.0,yes,no
44,2018-03-01,1519943010,4,1,14,14,1519942810,1.0,yes,no
45,2018-03-01,1519920307,5,1,15,15,1519918950,1.0,yes,no


In [55]:
#subtract to get minutes respone
#message_df_filtered['time_delta'] = round((message_df_filtered['timestamp'] - message_df_filtered['tt_lag'])/60, 1)

message_df_filtered.loc['time_delta'] = message_df_filtered.apply(lambda x: round((x.timestamp - x.tt_lag)/60, 1), axis=1)

/var/folders/rn/ljpx9hjx4yvftbhwbld6fwjc0000gn/T/ipykernel_1062/951476275.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  message_df_filtered.loc['time_delta'] = message_df_filtered.apply(lambda x: round((x.timestamp - x.tt_lag)/60, 1), axis=1)


In [57]:
message_df_filtered.head(5)

,date,timestamp,sender_id,receiver_id,key,key_lag,tt_lag,sender_lag,is_respone,same_sender_receiver,time_delta
47,2018-03-01,1.519921e+09,2.0,1.0,12,12,1.519920e+09,1.0,yes,no,2.4
3,2018-03-01,1.519930e+09,1.0,2.0,12,12,1.519921e+09,2.0,yes,no,159.3
46,2018-03-01,1.519930e+09,2.0,1.0,12,12,1.519930e+09,1.0,yes,no,3.9
44,2018-03-01,1.519943e+09,4.0,1.0,14,14,1.519943e+09,1.0,yes,no,3.3
45,2018-03-01,1.519920e+09,5.0,1.0,15,15,1.519919e+09,1.0,yes,no,22.6


In [60]:
#filter less than 5 mins
message_less_5 = message_df_filtered[(message_df_filtered['time_delta'] <= 5)]

In [61]:
round(len(message_less_5)/len(message_df),2)

0.21

In [75]:
mss_df = df.copy()

In [76]:
#use sender_id and receiver_id to create primary key
mss_df['key'] = np.where(mss_df['sender_id'] >= mss_df['receiver_id'],\
                            mss_df['receiver_id'].astype(str)+mss_df['sender_id'].astype(str),
                            mss_df['sender_id'].astype(str)+mss_df['receiver_id'].astype(str))

In [79]:
mss_df = mss_df.sort_values(by=['key', 'timestamp'])

In [114]:
print(mss_df.shape)
mss_df.head(10)

(47, 8)


,date,timestamp,sender_id,receiver_id,key,times_lag,receiver_lag,times_delta
4,2018-03-01,1519920410,1,2,12,1519920555,1.0,2.4
47,2018-03-01,1519920555,2,1,12,1519930114,2.0,159.3
3,2018-03-01,1519930114,1,2,12,1519930348,1.0,3.9
1,2018-03-01,1519942810,1,4,14,1519943010,1.0,3.3
2,2018-03-01,1519918950,1,5,15,1519920307,1.0,22.6
45,2018-03-01,1519920307,5,1,15,1519923378,5.0,51.2
0,2018-03-01,1519923378,1,5,15,1519923903,1.0,8.8
5,2018-03-01,1519868747,2,3,23,1519868758,2.0,0.2
13,2018-03-01,1519898112,3,2,23,1519898323,3.0,3.5
8,2018-03-01,1519899889,2,3,23,1519900495,2.0,10.1


In [89]:
mss_df['times_lag'] = mss_df['timestamp'].shift(-1)
mss_df = mss_df.fillna(0)
mss_df['times_lag'] = mss_df['times_lag'].astype(int)

In [92]:
mss_df['receiver_lag'] = mss_df['receiver_id'].shift(-1)

In [96]:
mss_df = mss_df[mss_df['receiver_lag'] == mss_df['sender_id']]
mss_df = mss_df[mss_df['sender_id'] != mss_df['receiver_id']]

In [108]:
mss_df['times_delta'] = round((mss_df['times_lag'] - mss_df['timestamp'])/60,1)

In [109]:
df_5 = mss_df[mss_df['times_delta'] <= 5]

In [110]:
df_5.shape

(19, 8)

In [113]:
round(len(df_5)/len(df),2)

0.22